# Load Packages

In [132]:
#!pip install polars

In [2]:
import bz2
import zstandard as zstd
import io
import re
import time
import polars as pl

# Cleaning the data

We keep only the games where both players have at leat 2400 elo.

In [3]:
#Example game 

# [Event "Rapid swiss https://lichess.org/swiss/7siKNYGN"]
# [Site "https://lichess.org/ECy8BFok"]
# [Date "2025.03.01"]
# [Round "-"]
# [White "STRI0006"]
# [Black "PORT0024"]
# [Result "1-0"]
# [UTCDate "2025.03.01"]
# [UTCTime "00:00:00"]
# [WhiteElo "939"]
# [BlackElo "404"]
# [WhiteRatingDiff "+3"]
# [BlackRatingDiff "-4"]
# [ECO "C41"]
# [Opening "Philidor Defense"]
# [TimeControl "600+0"]
# [Termination "Normal"]

# 1. e4 { [%eval 0.18] [%clk 0:10:00] } 1... e5 { [%eval 0.21] [%clk 0:10:00] } 2. Nf3 { [%eval 0.13] [%clk 0:09:59] } 2... d6 { [%eval 0.48] [%clk 0:10:00] } 3. Nc3 { [%eval 0.28] [%clk 0:09:57] } 3... Nc6 { [%eval 0.56] [%clk 0:09:52] } 4. Bc4 { [%eval 0.29] [%clk 0:09:55] } 4... Be6 { [%eval 0.42] [%clk 0:09:43] } 5. Bxe6 { [%eval 0.41] [%clk 0:09:53] } 5... fxe6 { [%eval 0.33] [%clk 0:09:42] } 6. d3 { [%eval 0.19] [%clk 0:09:49] } 6... d5? { [%eval 1.56] [%clk 0:09:29] } 7. exd5?! { [%eval 0.92] [%clk 0:09:48] } 7... Nb4? { [%eval 3.42] [%clk 0:09:02] } 8. dxe6 { [%eval 3.32] [%clk 0:09:42] } 8... Nxd3+? { [%eval 6.02] [%clk 0:08:59] } 9. cxd3 { [%eval 6.29] [%clk 0:09:40] } 9... Bb4 { [%eval 8.18] [%clk 0:08:46] } 10. Bg5 { [%eval 6.43] [%clk 0:09:36] } 10... Qd7?? { [%eval 14.09] [%clk 0:08:42] } 11. exd7+ { [%eval 13.67] [%clk 0:09:35] } 11... Kxd7 { [%eval 12.74] [%clk 0:08:40] } 12. Nxe5+ { [%eval 12.99] [%clk 0:09:34] } 12... Kc8?! { [%eval #4] [%clk 0:08:31] } 13. Qg4+ { [%eval #3] [%clk 0:09:30] } 13... Kb8 { [%eval #3] [%clk 0:08:24] } 14. O-O-O?! { [%eval 12.06] [%clk 0:09:22] } 14... b6 { [%eval 12.62] [%clk 0:08:21] } 15. Qxb4 { [%eval 12.49] [%clk 0:09:10] } 15... Nf6 { [%eval 12.61] [%clk 0:08:10] } 16. Nf7 { [%eval 11.93] [%clk 0:09:05] } 16... Rc8 { [%eval 12.35] [%clk 0:08:05] } 17. Bf4 { [%eval 12.26] [%clk 0:08:57] } 17... a5?! { [%eval #1] [%clk 0:07:58] } 18. Qe4?! { [%eval 10.01] [%clk 0:08:54] } 18... Nxe4 { [%eval 10.11] [%clk 0:07:49] } 19. dxe4 { [%eval 10.11] [%clk 0:08:53] } 19... g5 { [%eval 10.38] [%clk 0:07:42] } 20. Bxg5 { [%eval 10.05] [%clk 0:08:49] } 20... h5 { [%eval 10.31] [%clk 0:07:41] } 21. Bd2 { [%eval 9.51] [%clk 0:08:42] } 21... Ra6 { [%eval 10.13] [%clk 0:07:36] } 22. Bf4 { [%eval 10.14] [%clk 0:08:40] } 22... Kb7 { [%eval 10.32] [%clk 0:07:30] } 23. Rd8 { [%eval 9.93] [%clk 0:08:31] } 23... Rxd8 { [%eval 10.16] [%clk 0:07:27] } 24. Nxd8+ { [%eval 10.19] [%clk 0:08:30] } 24... Kc8 { [%eval 10.09] [%clk 0:07:23] } 25. Nc6 { [%eval 9.96] [%clk 0:08:26] } 25... Kb7 { [%eval 9.94] [%clk 0:07:20] } 26. Ne7 { [%eval 9.76] [%clk 0:08:19] } 26... h4 { [%eval 10.77] [%clk 0:07:16] } 27. e5 { [%eval 9.56] [%clk 0:08:06] } 27... c6 { [%eval 11.05] [%clk 0:07:13] } 28. Ng6 { [%eval 10.05] [%clk 0:07:52] } 28... a4 { [%eval 10.75] [%clk 0:07:06] } 29. e6 { [%eval 10.64] [%clk 0:07:49] } 29... a3 { [%eval 10.54] [%clk 0:07:02] } 30. bxa3 { [%eval 10.52] [%clk 0:07:48] } 30... Rxa3 { [%eval 10.15] [%clk 0:07:00] } 31. Be5 { [%eval 10.21] [%clk 0:07:43] } 31... Rxa2?! { [%eval #8] [%clk 0:06:57] } 32. Nxa2 { [%eval #7] [%clk 0:07:41] } 32... c5 { [%eval #7] [%clk 0:06:54] } 33. e7 { [%eval #6] [%clk 0:07:40] } 33... b5 { [%eval #4] [%clk 0:06:53] } 34. e8=Q { [%eval #3] [%clk 0:07:38] } 34... c4 { [%eval #3] [%clk 0:06:48] } 35. Qxb5+ { [%eval #2] [%clk 0:07:36] } 35... Ka7 { [%eval #2] [%clk 0:06:46] } 36. Qb8+ { [%eval #2] [%clk 0:07:31] } 36... Ka6 { [%eval #2] [%clk 0:06:43] } 37. Nb4+ { [%eval #2] [%clk 0:07:28] } 37... Ka5 { [%eval #2] [%clk 0:06:41] } 38. Bc7+ { [%eval #2] [%clk 0:07:25] } 38... Ka4 { [%eval #2] [%clk 0:06:39] } 39. Nc2 { [%eval #1] [%clk 0:07:22] } 39... c3 { [%eval #1] [%clk 0:06:35] } 40. Bd6 { [%eval #2] [%clk 0:07:20] } 40... h3 { [%eval #1] [%clk 0:06:32] } 41. Qb4# { [%clk 0:07:19] } 1-0

In [4]:
# Configuration constants
input_path = "lichess_db_standard_rated_2025-03.pgn.zst"
output_path = "games_clean.csv"
min_elo = 2400

csv_headers = "ID;Result;WhiteElo;BlackElo;ECO;Opening;TimeControl;Termination;Moves\n"
keep = ("[Site ", "[Result ", "[WhiteElo ", "[BlackElo ", 
        "[ECO ", "[Opening ", "[TimeControl ", "[Termination ", "1.")

In [136]:
def extract_value(line):
    """Extract the quoted value from a PGN header line."""
    # e.g. '[White "STRI0006"]' -> 'STRI0006'
    return line.split("\"")[1]

In [137]:
def clean_moves(line):
    """Strip the moves of annotation"""
    #e.g. 1. e4 { [%eval 0.18] [%clk 0:10:00] } 1... e5 { [%eval 0.21] [%clk 0:10:00] } 2. Nf3 { [%eval 0.48] [%clk 0:10:00] } 1-0
    #–> 1. e4 e5 2. Nf3 1-0
    line = re.sub(r"\s\{[^{}]*\}|[!?]+", "", line) #when I see {xxxxx} or ! and ? remove it
    return re.sub(r'\s+\d+\.\.\.\s+', ' ', line).strip() #delete 2...

In [ ]:
start = time.time()
with open(input_path, "rb") as compressed_file:
    dctx = zstd.ZstdDecompressor()
    with dctx.stream_reader(compressed_file) as reader:
        text_stream = io.TextIOWrapper(reader, encoding="utf-8")
    
        game_count = 0
        loaded_count = 0
        game = ""

        with open(output_path, "w") as out_f:
            out_f.write(csv_headers)
            
            for line in text_stream:
                if not line.startswith(keep):
                    continue

                #we mark the beginning of the game
                elif line.startswith("[Site"):
                    game_count += 1 #how many games I checked
                    if game_count % 100000 == 0:
                        print(f"Checked games: {game_count}, loaded {loaded_count}, time {(time.time() - start):.2f} seconds")

                    #If the game string is non empty (the game passed the criteria) we load the game
                    if game:
                        out_f.write(game + "\n")
                        game = "" #empty the game after loading

                        loaded_count += 1
                    
                    #Initialize the game with the ID
                    game = extract_value(line).split("/")[-1]
        
                elif game:
                    if line.startswith("[WhiteElo"):
                        #check white elo
                        w_elo = extract_value(line)
                        if int(w_elo) >= min_elo:
                            #if the condition is satisfied, check black elo
                            next_line = next(text_stream)
                            b_elo = extract_value(next_line)

                            if int(b_elo) >= min_elo:
                                game += ";" + w_elo + ";" + b_elo
                            else:
                                game = ""
                        else:
                            game = ""
                    elif line.startswith("1."):
                        game += ";" + clean_moves(line)
                    else:
                        game += ";" + extract_value(line)
            if game: #load the last game if non empty
                loaded_count += 1
                out_f.write(game + "\n")
                
        print(f"Finished. Total checked: {game_count}, loaded: {loaded_count}, time: {(time.time() - start):.2f} seconds")  
        #Finished. Total checked: 97512351, loaded: 2015001, time: 3325.78 seconds          

Checked games: 100000, loaded 2001, time 5.17 seconds
Checked games: 200000, loaded 4149, time 8.21 seconds
Checked games: 300000, loaded 6236, time 11.00 seconds
Checked games: 400000, loaded 8242, time 13.49 seconds
Checked games: 500000, loaded 10127, time 16.33 seconds
Checked games: 600000, loaded 12009, time 19.11 seconds
Checked games: 700000, loaded 13891, time 21.61 seconds
Checked games: 800000, loaded 15954, time 24.19 seconds
Checked games: 900000, loaded 18069, time 27.96 seconds
Checked games: 1000000, loaded 19990, time 32.83 seconds
Checked games: 1100000, loaded 21944, time 37.92 seconds
Checked games: 1200000, loaded 23883, time 41.69 seconds
Checked games: 1300000, loaded 25779, time 48.18 seconds
Checked games: 1400000, loaded 27535, time 53.13 seconds
Checked games: 1500000, loaded 29402, time 56.11 seconds
Checked games: 1600000, loaded 31104, time 59.53 seconds
Checked games: 1700000, loaded 32985, time 62.10 seconds
Checked games: 1800000, loaded 34884, time 64.

In [5]:
df = pl.read_csv(output_path, separator = ";")

# Further Cleaning

## Transform the time control column

In [ ]:
#time + 40* (increment)
# < 29s = UltraBullet
# < 179s = Bullet
# < 479s = Blitz
# < 1499s = Rapid
# ≥ 1500s = Klasický šach
def time_control(x):
    """ Assigns the time control name to time control"""
    if "+" in x:
        time, inc = [int(i) for i in x.split("+")]
        sum_time = time + 40*inc
        if sum_time < 29:
            return "UltraBullet"
        elif sum_time < 179:
            return "Bullet"
        elif sum_time < 479:
            return "Blitz"
        elif sum_time < 1499:
            return "Rapid"
        else:
            return "Classic"
    else:
        return "NoTime"

In [ ]:
df = df.with_columns(pl.col("TimeControl").map_elements(lambda x: time_control(x), return_dtype=str))
df.head(5)


ID,Result,WhiteElo,BlackElo,ECO,Opening,TimeControl,Termination,Moves
str,str,i64,i64,str,str,str,str,str
"""q2RE2Q1D""","""1/2-1/2""",2409,2502,"""D04""","""Queen's Pawn Game: Colle Syste…","""Bullet""","""Time forfeit""","""1. d4 d5 2. e3 Nf6 3. Nf3 c5 4…"
"""UzdH90ui""","""0-1""",2468,2419,"""A43""","""Benoni Defense: Old Benoni""","""Bullet""","""Time forfeit""","""1. d4 c5 2. Nc3 cxd4 3. Qxd4 N…"
"""OghDdxVv""","""0-1""",2451,2460,"""E64""","""King's Indian Defense: Fianche…","""Bullet""","""Normal""","""1. d4 Nf6 2. c4 g6 3. Nf3 Bg7 …"
"""5LceqUv2""","""0-1""",2790,2683,"""B00""","""Owen Defense""","""Bullet""","""Normal""","""1. e4 b6 2. d4 Bb7 3. Bd3 f5 4…"
"""prTpr3m1""","""0-1""",2498,2502,"""A10""","""English Opening: Anglo-Scandin…","""Bullet""","""Time forfeit""","""1. c4 d5 2. cxd5 Qxd5 3. Nc3 Q…"


In [13]:
#check the result
freq_time = df["TimeControl"].value_counts(sort = True)
freq_time

TimeControl,count
str,u32
"""Bullet""",1638752
"""Blitz""",346194
"""Rapid""",22891
"""UltraBullet""",4679
"""Classic""",2353
"""NoTime""",283


## Remove Unterminated, Rules infraction and Abandoned games

In [143]:
df = df.filter(
    ~pl.col("Termination").is_in(["Unterminated", "Rules infraction", "Abandoned"])
)

In [12]:
#check the result
freq_term = df["Termination"].value_counts(sort = True)
freq_term

Termination,count
str,u32
"""Normal""",1309263
"""Time forfeit""",705889


## Clean Moves & Keep the games with at least 5 moves

In [14]:
df = df.filter(
    ~pl.col("Moves").is_null())

In [ ]:
def min_game_length(game_str, min):
    '''checks if the game has at least min moves'''
    if str(min) + "." in game_str:
        return True
    return False

In [16]:
mask = [min_game_length(game, min = 5) for game in df["Moves"]]
df = df.filter(mask)

In [19]:
def remove_result(game_str):
    '''Removes the result from the game string'''
    tokens = game_str.strip().split(" ")
    if tokens[-1] in {'1-0', '0-1', '1/2-1/2', '*'}:
        tokens = tokens[:-1]
    return ' '.join(tokens)

In [22]:
df = df.with_columns(pl.col("Moves").map_elements(remove_result, return_dtype=str))

In [23]:
df.write_csv(output_path, separator = ";")